# Fresh Idea
## separate one/zero activity of domains
- replace zeros by minus one
- calculate the class activity for 3 hours bins for each domain
- calculate the user activity for gaussian around center of 3 hour bins
- calculate the likelihood of the person being a 1/-1 in that time 
- add user general metrics including domain cls activity and usage patterns

### questions
- how to take into account times when the person used a website when others didnt?
- how to give likelihood when the person didn't show any nearby activity?
- what if he used a similar website at same time but more nich? 
- how to average the bins weighted by the significance of that bin?
- how to give weight to the magnitude of number of users entering? probability of 1 with confidence
- what about sparse websites?
- how to not let times where there are no activity take a lot of weight?
### enhancements
- create graph embedding of urls
- for each bin, calculate the metric per url
- instead of only looking at the specific website, take into account websites with similar usages,
  for example looking at same domain_cls usage in gaussian around bin, or looking at domain embeddings and looking at the activity in similar embeddings weighted by the distance in the embedding space

### NOTICE:
the data itself will use all domains, even ones that the person never used. this could be an issue. 
first of all the fact that the person doesnt use them is an indication. we 
- we might want to take the niche websites and sum them up
- we might want to remove them

IDEA!
- use different features for different people
- make an ensemble that can differentiate between different users
- take the people that get a wrong prediction and see if a classifier that is more "fringe" can classify them better
- for example another tree classifier that takes a smaller amount of features to give more opportunity to fringe websites
can create a classifier for each user type 
can take number of usages for each domain, and cluster people or PCA
clustering is good - I can create a classifier for each cluster, from each cluster take all of the available data for all of the visited domains, and create a classifier for them. use only data from those users or all users that used one of the websites, plus the general model, for each cluster - use cluster model and general model.
also - I can multiply the features by the log of usages

another idea is to simply average the most prominent websites weighted by the specific user usages, and the general usage

USER CLUSTER AS FEATURE - or PCA coefficients


In [1]:

import os
import ray
from modin.db_conn import ModinDatabaseConnection
from modin.config import NPartitions,RangePartitioning
os.environ["NEPTUNE_API_TOKEN"] = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJiMGMyZjIyZC0xMjQzLTQxNjQtYjZjZC0wMTRiZmJmZmRlZjYifQ=="
    # !export MODIN_CPUS=2
# n_cpus = 8
plasma_store_size = 100*(1024**3)
heap_memory = 210*(1024**3)
# os.environ["MODIN_CPUS"] = str(n_cpus)
# os.environ["MODIN_ENGINE"] = "ray"
os.environ["MODIN_NPARTITIONS"] = "24"
os.environ["MODIN_RANGE_PARTITIONING"] = "True"
# os.environ["MODIN_MEMORY"] = str(plasma_store_size)
ray.init(num_cpus =22,ignore_reinit_error=True, object_store_memory=plasma_store_size,_memory=heap_memory)
# print(ray.cluster_resources())
import modin.pandas as mpd
from modin import config as cfg
print(vars(cfg))
NEPTUNE_MODE="sync"


2025-03-23 01:25:00,315	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


{'__name__': 'modin.config', '__doc__': 'Module houses config entities which can be used for Modin behavior tuning.', '__package__': 'modin.config', '__loader__': <_frozen_importlib_external.SourceFileLoader object at 0x7f1814220450>, '__spec__': ModuleSpec(name='modin.config', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f1814220450>, origin='/home/tom.touati/mafat-challenge/.venv/lib/python3.11/site-packages/modin/config/__init__.py', submodule_search_locations=['/home/tom.touati/mafat-challenge/.venv/lib/python3.11/site-packages/modin/config']), '__path__': ['/home/tom.touati/mafat-challenge/.venv/lib/python3.11/site-packages/modin/config'], '__file__': '/home/tom.touati/mafat-challenge/.venv/lib/python3.11/site-packages/modin/config/__init__.py', '__cached__': '/home/tom.touati/mafat-challenge/.venv/lib/python3.11/site-packages/modin/config/__pycache__/__init__.cpython-311.pyc', '__builtins__': {'__name__': 'builtins', '__doc__': "Built-in functions, exceptions,

In [2]:
# !export MODIN_CPUS=2
import neptune
import os

run = neptune.init_run(
    project="tom.touati/web-segmentation",  # replace with your project
    api_token=os.environ["NEPTUNE_API_TOKEN"],
    # name="Activity-Based Features",
    capture_stdout=True,
    capture_stderr=True,
    capture_hardware_metrics=True,
    tags=["time-based-models", "activity-based-features"],
    description="User activity patterns analysis",
    mode=NEPTUNE_MODE
)
import neptune

# Initialize Neptune run

# Log parameters
params = {
    "training_data":{
        "min_domain_devices": 10,
        "n_devices_hist":True,
        "test_size": 0.2,
        "random_state": 42
    },
    "user_activity_timeseries": {
        "bin_hours": 6,
        "gaussian_filter": True, 
        "n_days_each_side": 3,
        "std": 1.5,
        "drop_na": True,
        "drop_zeros": False
    },
    "domain_activity_timeseries": {
        "bin_hours": 6,
        "gaussian_filter": True, 
        "n_days_each_side": 7,
        "std": 1.5,
        "drop_na": False,
        "drop_zeros": True
    },
    "general_user_time_bin": {
        "should_run": True,
        "bin_hours": 3
    },
    "feature_selection": {
        "n_features": 1000,
        "step": 20000
    },
    "model": {
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "learning_rate": 0.1,
        "n_estimators": 150,
        "max_depth": 6
    }
}

run["parameters"] = params

# Log notebook
run["notebook"].upload("/home/tom.touati/mafat-challenge/code/time_based_model.ipynb")

# Log metrics


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/tom.touati/web-segmentation/e/WEB-56


[neptune] [warning] NeptuneWarning: Info (NVML): NVML Shared Library Not Found. GPU usage metrics may not be reported. For more information, see https://docs.neptune.ai/help/nvml_error/


In [3]:
import os
import sqlite3
# %matplotlib widget
import matplotlib.pyplot as plt
from multiprocessing import freeze_support

%load_ext autoreload
%autoreload 2
  # Modin will use Ray
# ray.init()
# NPartitions.put(16)
def load_data_from_db(con):
    try:
        # First get 1000 random Device_IDs
        # selective_device_ids_query = """
        # WITH random_devices AS (
        #     SELECT DISTINCT Device_ID 
        #     FROM data 
        #     LIMIT 1000
        # )
        # SELECT Domain_Name, Device_ID, Target,Datetime
        # FROM data 
        # WHERE Device_ID IN (SELECT Device_ID FROM random_devices)
        # AND Domain_Name != 1732927
        # """
        device_ids_query = """SELECT Domain_Name, Device_ID, Target,Datetime from data"""
        # WHERE Domain_Name != 1732927 """
        df = mpd.read_sql(device_ids_query, con
                         )._repartition()
        df = df[df['Domain_Name'] != 1732927]
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        raise


# Add this line

In [ ]:
# load data from db
freeze_support()
dbfile = '../../data/training_set.db'

conn = ModinDatabaseConnection('sqlalchemy', f'sqlite:///{dbfile}')

# Can use get_connection to get underlying sqlalchemy engine
conn.get_connection()
db_df = load_data_from_db(conn)
print(db_df.head())
del conn  

In [5]:
db_df['Datetime'] = mpd.to_datetime(db_df['Datetime'])
db_df.set_index('Datetime', inplace=True)
db_df = db_df.astype( {'Domain_Name': 'uint32', 'Device_ID': 'uint32', 'Target': 'uint8'})

In [6]:
# prepare training data
import matplotlib.pyplot as plt
#train test split
from sklearn.model_selection import train_test_split
def get_train_test_devices(device_target_df, test_size=0.2, random_state=42):    
    # Perform stratified split on device IDs
    train_device_ids, test_device_ids = train_test_split(
        device_target_df['Device_ID'],
        test_size=test_size,
        random_state=random_state,
        stratify=device_target_df['Target']
    )
    return train_device_ids, test_device_ids

def get_initial_train_data(db_df, test_size=0.2, random_state=42, min_domain_devices=10,n_devices_hist=False):
    device_targets = db_df.groupby("Device_ID")["Target"].first().reset_index()
    train_devices, test_device_ids = get_train_test_devices(device_targets,test_size=test_size,random_state=random_state)
    train_df = db_df[db_df["Device_ID"].isin(train_devices)]
    devices_per_domain = train_df.groupby("Domain_Name")["Device_ID"].nunique()
    
    domain_mask = devices_per_domain>min_domain_devices
    print(f"Percentage of domains with more than {min_domain_devices} devices: {domain_mask.mean()*100:.2f}%")
    devices_per_domain = devices_per_domain[domain_mask]
    if n_devices_hist:
        hist = devices_per_domain.hist()
        run["plots/domain_devices_hist"].upload(neptune.types.File.as_image(hist.figure))
        plt.show()
    train_df = train_df[train_df["Domain_Name"].isin(devices_per_domain.index)]
    return train_df,train_devices,test_device_ids, device_targets, devices_per_domain


In [ ]:
train_df,train_devices,test_device_ids, device_targets,devices_per_valid_domain = get_initial_train_data(db_df,**params["training_data"])


In [8]:
# %%writefile timeseries_processing.py
# preprocess timeseries
from functools import partial
def process_activity_timeseries(domain_df,bin_hours=6,gaussian_filter=True,n_days_each_side=3,std=1.5,drop_na=True,drop_zeros=False):
    activity_per_3h = domain_df[["Device_ID"]].resample(f'{str(bin_hours)}h').nunique()
    activity_per_3h.rename(columns={"Device_ID":"Activity"},inplace=True)
    # activity_per_3h = activity_per_3h.to_frame()

    gaussian_window_hours = int(n_days_each_side*24/bin_hours*2) # n_days_each_side * 24h / 3h_per_bin * 2 sides
    if gaussian_filter:
        activity_per_3h = activity_per_3h.rolling(window=gaussian_window_hours, win_type='gaussian',center=True,min_periods=1,closed="both").mean(std=std)
    if drop_na:
        activity_per_3h.dropna(inplace=True)
    if drop_zeros:
        activity_per_3h = activity_per_3h[activity_per_3h["Activity"]!=0]
    return activity_per_3h.round().astype(int)

def get_domain_activity_timeseries(train_df,domain_ts_kwargs):
    process_domain_timeseries = partial(process_activity_timeseries,**domain_ts_kwargs)
    process_domain_timeseries.__name__ =process_activity_timeseries.__name__
    domain_timeseries = train_df[["Domain_Name","Target","Device_ID"]].groupby(["Domain_Name","Target"]).apply(process_domain_timeseries)
    
    domain_timeseries = domain_timeseries
    domain_timeseries["activity_fraction"] = domain_timeseries.groupby(["Domain_Name", "Target"]).transform(lambda x: x/x.sum())
    # Add the sum of activity as a new column
    domain_activity = domain_timeseries.groupby(["Domain_Name", "Target"])[["Activity"]].sum()

    domain_activity = domain_activity.rename(columns={"Activity": "target_domain_activity"})
    # Merge the results
    domain_timeseries = domain_timeseries.merge(domain_activity, left_index=True, right_index=True)
    # Reset index to get Target as a column, then pivot to get Target as columns
    pivot_fraction_ts = domain_timeseries.reset_index().pivot(
        index=['Datetime', 'Domain_Name'],
        columns='Target'
    ).fillna(0)
    pivot_fraction_ts.columns = [f'{col[0]}_{col[1]}' for col in pivot_fraction_ts.columns]
    return pivot_fraction_ts
def get_user_activity_timeseries(db_df,user_ts_kwargs):
    process_user_timeseries = partial(process_activity_timeseries,**user_ts_kwargs)
    process_user_timeseries.__name__ =process_activity_timeseries.__name__
    user_timeseries = db_df[["Domain_Name","Device_ID"]].groupby(["Domain_Name","Device_ID"]).apply(process_user_timeseries)
    # user_timeseries.index = mpd.MultiIndex.from_tuples(user_timeseries.index.to_list(),names=["Domain_Name","Device_ID","Datetime"])
    return user_timeseries

In [ ]:
# train_df = train_df._repartition()
domain_activity_timeseries = get_domain_activity_timeseries(
    train_df._repartition(), params["domain_activity_timeseries"])


In [62]:
# best_domain_activity = domain_activity_timeseries.reset_index("Datetime").loc[best_features].set_index("Datetime",append=True)

In [ ]:
user_activity_timeseries = get_user_activity_timeseries(
    db_df.loc[db_df["Domain_Name"].isin(devices_per_valid_domain.index)], params["user_activity_timeseries"])


In [19]:
import gc
import ctypes
import sys

def cleanup_memory():
    """
    Force cleanup of memory by:
    1. Running garbage collection
    2. Attempting to release memory back to OS
    """
    # Force garbage collection
    gc.collect()
    
    # Attempt to release memory back to the OS
    if sys.platform.startswith('linux'):
        libc = ctypes.CDLL('libc.so.6')
        # MALLOC_TRIM(0) releases memory back to OS if possible
        libc.malloc_trim(0)

# Clean up memory
cleanup_memory()

In [20]:
# from scipy import stats
import numpy as np

def class_probability_score(active, p_active_given_a, p_active_given_b, prior_a=0.5, total_users=100):
    """
    Calculate class probability score with vectorized operations
    
    Args:
        active: Boolean indicating if user was active
        p_active_given_a: Probability of activity given class A (0)
        p_active_given_b: Probability of activity given class B (1)
        prior_a: Prior probability for class A
        total_users: Total number of users for confidence calculation
    """
    # Use numpy for vectorized operations
    likelihood_a = np.where(active, p_active_given_a, 1 - p_active_given_a)
    likelihood_b = np.where(active, p_active_given_b, 1 - p_active_given_b)
    
    # Avoid division by zero
    evidence = (likelihood_a * prior_a + likelihood_b * (1 - prior_a))
    posterior_a = (likelihood_a * prior_a) / evidence

    return posterior_a #* confidence_factor

def get_user_domain_scores(domain_activity_timeseries,user_activity_timeseries):
    merged_timeseries_df = domain_activity_timeseries.reset_index().merge(
        user_activity_timeseries.reset_index(), how="left", on=["Domain_Name", "Datetime"]
    ).set_index(["Datetime", "Domain_Name", "Device_ID"])

    merged_timeseries_df["bin_activity"] = merged_timeseries_df["Activity_0"]+merged_timeseries_df["Activity_1"]
    merged_timeseries_df["total_activity"] = (merged_timeseries_df["target_domain_activity_0"]+merged_timeseries_df["target_domain_activity_1"])
    merged_timeseries_df["relative_0_activity"] = merged_timeseries_df["target_domain_activity_0"]/merged_timeseries_df["total_activity"]
    merged_timeseries_df["score"]=class_probability_score(merged_timeseries_df["Activity"], merged_timeseries_df["activity_fraction_0"], merged_timeseries_df["activity_fraction_1"], prior_a=merged_timeseries_df["relative_0_activity"], total_users=merged_timeseries_df["bin_activity"])
    merged_timeseries_df["weighted_score"] = (merged_timeseries_df["score"])*(merged_timeseries_df["bin_activity"])#np.log(1+merged_df["bin_activity"]).astype(int))
    final_scores = merged_timeseries_df.groupby(["Device_ID","Domain_Name"])["weighted_score"].mean()
    final_scores_pivot = final_scores.to_frame().reset_index().pivot(index="Device_ID",columns="Domain_Name").fillna(0)

    final_scores_pivot=(final_scores_pivot-final_scores_pivot.values.min())/(final_scores_pivot.values.max()-final_scores_pivot.values.min())*2-1
    return final_scores_pivot

In [ ]:
final_scores_pivot = get_user_domain_scores(domain_activity_timeseries,user_activity_timeseries)

In [22]:
from sklearn.decomposition import PCA

# baseline features
def get_active_days_per_user(user_domain_ts):
    """
    Calculate the number of unique days each user had any activity.

    Args:
        user_domain_ts: MultiIndex Series with levels [Domain_Name, Device_ID, Datetime]

    Returns:
        Series with index Device_ID and values being number of unique active days
    """
    # Reset index to get Datetime as a column
    df = user_domain_ts.reset_index()

    # Convert Datetime to date (removing time component)
    df['Date'] = df['Datetime'].dt.date

    # Group by Device_ID and count unique dates where Activity > 0
    active_days = df[df['Activity'] > 0].groupby('Device_ID')['Date'].nunique()

    active_days = active_days.astype(int)
    active_days.name = "Active_Days"
    active_days = (active_days-active_days.min()
                   ) / (active_days.max()-active_days.min())*2-1
    return active_days


def get_activity_per_time_bin(df, bin_hours=3):
    # Convert datetime to time only
    # time_index = db_df.index.to_series().dt.time
    # df["time"] = time_index
    df_copy = df.copy()
    df_copy["time"] = db_df.index.to_series().dt.hour.astype(int)//bin_hours
    df_copy["day_part_activity"] = 0
    activity_per_time_range = df_copy[["Device_ID", "time", "day_part_activity"]].groupby(
        ["Device_ID", "time"]).count()
    activity_per_time_range["activity_fraction"] = activity_per_time_range.groupby("Device_ID").apply(lambda x: x/x.sum()).values
    activity_per_time_range = activity_per_time_range[["activity_fraction"]].reset_index()
    activity_per_time_range = activity_per_time_range.pivot(index="Device_ID",columns="time",values="activity_fraction")
    activity_per_time_range.columns = [f"time_{col}" for col in activity_per_time_range.columns]
    activity_per_time_range = (activity_per_time_range-activity_per_time_range.stack().min())/(activity_per_time_range.stack().max()-activity_per_time_range.stack().min())*2-1
    activity_per_time_range = activity_per_time_range.fillna(0)
    return activity_per_time_range  # .round().astype(int)
def get_device_domain_pca(user_activity_timeseries, n_components=100):
    # Calculate total entries per domain for each device
    domain_entries = user_activity_timeseries.reset_index()[['Device_ID', 'Domain_Name','Activity']].groupby(['Device_ID', 'Domain_Name'])['Activity'].sum()
    domain_entries_pivot = domain_entries.unstack(fill_value=0)
    # domain_entries_pivot = domain_entries_pivot/domain_entries_pivot.sum(axis=1)
    # Normalize the data
    # normalized_data = (domain_entries_pivot - domain_entries_pivot.min().min()) / (domain_entries_pivot.max().max() - domain_entries_pivot.min().min())
    
    # Apply PCA
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(domain_entries_pivot.fillna(0))
    # Print explained variance ratio
    explained_variance = pca.explained_variance_ratio_
    print(f"Explained variance ratio: {explained_variance.sum():.3f}")
    # Create DataFrame with PCA results
    pca_df = mpd.DataFrame(
        pca_result,
        index=domain_entries_pivot.index,
        columns=[f'pca_domain_{i}' for i in range(n_components)]
    )
    
    return pca_df

In [ ]:
user_activity_timeseries

In [24]:
# active_days = get_active_days_per_user(user_activity_timeseries._repartition())
# # active_days = None
# # del user_activity_timeseries
# activity_per_time_range = get_activity_per_time_bin(db_df)


In [ ]:
# device_domain_PCA = get_device_domain_pca(user_activity_timeseries.fillna(0),n_components=10)
# device_domain_PCA

In [57]:
final_features = device_targets.set_index("Device_ID").join(final_scores_pivot)
# final_features = final_features.join(device_domain_PCA,how="left")
# if active_days is not None:
#     final_features = final_features.join(active_days,how="left")
# if activity_per_time_range is not None:
#     final_features = final_features.join(activity_per_time_range,how="left")
# final_features = final_features.fillna(0)
# final_features.columns = [str(col) for col in final_features.columns]

In [58]:
import xgboost
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score
def prepare_model_data(final_features,train_devices,test_device_ids):
    X_train = final_features[final_features.index.isin(train_devices)].drop('Target', axis=1)
    y_train = final_features[final_features.index.isin(train_devices)]['Target']

    X_test = final_features.loc[final_features.index.isin(test_device_ids)]
    y_test = final_features[final_features.index.isin(test_device_ids)]['Target']
    return X_train,y_train,X_test,y_test

def train_model(X_train,y_train,X_test,y_test,params):
    xgb_reg = xgboost.XGBRegressor(random_state=0, subsample=0.8, colsample_bytree=0.8, learning_rate= 0.1,
                               n_estimators= 150, max_depth=6, objective ='binary:logistic' ,eval_metric =roc_auc_score)
    selector = RFE(xgb_reg, n_features_to_select=1000, step=20000)
    selector = selector.fit(X_train, y_train)
    best_features = list(X_train.columns[selector.support_])
    test_prediction = selector.estimator_.predict(X_test[best_features])
    test_auc =round(roc_auc_score(y_test,test_prediction), 3)
    return test_auc,selector, best_features


In [ ]:
final_features.columns = [str(col) for col in final_features.columns]
X_train,y_train,X_test,y_test = prepare_model_data(final_features,train_devices,test_device_ids)
score,selector,best_features = train_model(X_train,y_train,X_test,y_test,params["model"])
print(f'The auc for validation set: {score}')
run["metrics/roc_auc"] = score
run["metrics/selected_features"] = str(best_features)
run["metrics/feature_importances"] = str(selector.estimator_.feature_importances_)
run["metrics/feature_ranking"] = str(selector.ranking_)
run["metrics/feature_support"] = str(selector.support_)
# Close the run

run.stop()

In [30]:
import json
with open("submission/best_features.json", "w") as fp:
    json.dump(best_features, fp)

In [31]:
mpd.Series(best_features).to_csv("selected_features.csv",index=False)

In [ ]:
import os
os.makedirs("submission",exist_ok=True)
# with open("submission/XGB_model.json", "w") as fp:
selector.estimator_.save_model('submission/XGB_model.json')

In [ ]:
domain_activity_timeseries[domain_activity_timeseries["Domain_Name"].isin(best_features)].to_parquet("domain_activity_timeseries.parquet")

In [36]:
# manual feature filter
# Create function to filter features based on mean values
# def filter_low_mean_features(features_df, percentile=0.1):
#     # Calculate absolute mean values for each feature
#     abs_means = abs(features_df).mean()
    
#     # Calculate percentile threshold of absolute means
#     threshold = abs_means.quantile(percentile)
    
#     # Get features with absolute means above threshold
#     significant_features = abs_means[abs_means >= threshold].index
    
#     # Filter features

#     return significant_features

# Apply the filtering function


In [ ]:
%%writefile model.py

import xgboost
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

class model:
    def __init__(self,with_neptune=False):
        '''
        Init the model
        '''
        self.model = xgboost.XGBRegressor(
            seed=0, 
            subsample=0.8, 
            colsample_bytree=0.8, 
            learning_rate=0.1,
            n_estimators=150, 
            max_depth=6, 
            objective='binary:logistic'
        )
        self.domain_activity = None
        self.user_activity = None
        self.best_features = None
        if with_neptune:
            self.init_neptune()

    def init_neptune(self,tags=["time-based-models", "activity-based-features"]):
        self.run = neptune.init(
                project="tom.touati/web-segmentation",  # replace with your project
                api_token=os.environ["NEPTUNE_API_TOKEN"],
                capture_stdout=True,
                capture_stderr=True,
                capture_hardware_metrics=True,
                tags=tags,
                description="User activity patterns analysis",
                mode=NEPTUNE_MODE
            )
    def process_activity_timeseries(self, domain_df, bin_hours=6, gaussian_filter=True, 
                                  n_days_each_side=3, std=1.5, drop_na=True, drop_zeros=False):
        activity_per_3h = domain_df[["Device_ID"]].resample(f'{str(bin_hours)}h').nunique()
        activity_per_3h.rename(columns={"Device_ID":"Activity"}, inplace=True)

        gaussian_window_hours = int(n_days_each_side*24/bin_hours*2)
        if gaussian_filter:
            activity_per_3h = activity_per_3h.rolling(
                window=gaussian_window_hours, 
                win_type='gaussian',
                center=True,
                min_periods=1,
                closed="both"
            ).mean(std=std)
            
        if drop_na:
            activity_per_3h.dropna(inplace=True)
        if drop_zeros:
            activity_per_3h = activity_per_3h[activity_per_3h["Activity"]!=0]
            
        return activity_per_3h.round().astype(int)

    def class_probability_score(self, active, p_active_given_a, p_active_given_b, prior_a=0.5):
        likelihood_a = np.where(active, p_active_given_a, 1 - p_active_given_a)
        likelihood_b = np.where(active, p_active_given_b, 1 - p_active_given_b)
        
        evidence = (likelihood_a * prior_a + likelihood_b * (1 - prior_a))
        posterior_a = (likelihood_a * prior_a) / evidence

        return posterior_a

    def get_user_domain_scores(self, user_timeseries, domain_activity):
        merged_df = domain_activity.reset_index().merge(
            user_timeseries.reset_index(), 
            how="left", 
            on=["Domain_Name", "Datetime"]
        ).set_index(["Datetime", "Domain_Name", "Device_ID"])

        merged_df["bin_activity"] = merged_df["Activity_0"] + merged_df["Activity_1"]
        merged_df["total_activity"] = (merged_df["target_domain_activity_0"] + merged_df["target_domain_activity_1"])
        merged_df["relative_0_activity"] = merged_df["target_domain_activity_0"]/merged_df["total_activity"]
        
        merged_df["score"] = self.class_probability_score(
            merged_df["Activity"], 
            merged_df["activity_fraction_0"], 
            merged_df["activity_fraction_1"], 
            prior_a=merged_df["relative_0_activity"]
        )
        
        merged_df["weighted_score"] = merged_df["score"] * merged_df["bin_activity"]
        final_scores = merged_df.groupby(["Device_ID","Domain_Name"])["weighted_score"].mean()
        final_scores_pivot = final_scores.to_frame().reset_index().pivot(
            index="Device_ID",
            columns="Domain_Name"
        ).fillna(0)

        final_scores_pivot = (final_scores_pivot-final_scores_pivot.values.min())/(
            final_scores_pivot.values.max()-final_scores_pivot.values.min())*2-1
            
        return final_scores_pivot

    def load(self, dir_path):
        '''
        Load the trained model and domain activity data
        '''
        import os
        import json
        model_path = os.path.join(dir_path, 'XGB_model.json')
        self.model.load_model(model_path)
        best_features_path = os.path.join(dir_path, 'selected_features.json')
        with open(best_features_path, "r") as fp:
            self.best_features = json.load(fp)
        domain_activity_path = os.path.join(dir_path, 'best_domain_activity.parquet')
        self.domain_activity = pd.read_parquet(domain_activity_path)

    def predict(self, X):
        '''
        Predict the class probability for the input data
        '''
        # Process user timeseries
        X = X.copy()
        X = X[X['Domain_Name'].isin(self.best_features)]
        X['Datetime'] = pd.to_datetime(X['Datetime'])
        X["Device_ID"] = 1
        X.set_index(['Datetime'], inplace=True)
        
        user_timeseries = X[['Device_ID', 'Domain_Name']].groupby(["Device_ID","Domain_Name"]).apply(lambda x :self.process_activity_timeseries(
            x,
            bin_hours=6,
            gaussian_filter=True,
            n_days_each_side=3,
            std=1.5,
            drop_na=True,
            drop_zeros=False
        ))
        
        # Get domain scores
        final_scores = self.get_user_domain_scores(user_timeseries, self.domain_activity)
        
        # Make prediction
        prediction = self.model.predict(final_scores)
        return prediction[0]

In [ ]:
import pandas as pd
X = pd.read_csv('test_sample.csv')
from model import *
M = model()
M.load('submission')
Y_test=[]
unique_Device_IDs = list(set(X.Device_ID))
for id in unique_Device_IDs:
    X_test = X.loc[X['Device_ID'] == id]
    X_test.drop('Device_ID', axis=1, inplace=True)
    Y_test.append(M.predict(X_test))

print(f'Prediction: {Y_test[0]}')